In [5]:
import copy
import torch
import torch.nn as nn
import torch.optim as optim

from DataPreprocess import create_dataloader
from InitializingModule import InitModel
from TrainingAlgorithm import train

In [2]:
train_loader, val_loader = create_dataloader(datapath='data/SPECT_data.txt', batch_size=128)

In [3]:
device = torch.device('cuda')
a = InitModel(input_size=22, hidden_size=3, output_size=1, device=device, batch_norm=True)
test_model = a.init_module_multi_ReLU_AE(train_loader)
optimizer = optim.Adam(test_model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

In [6]:
saved_model = copy.deepcopy(test_model)

In [94]:
train_loader.dataset[3][1].unsqueeze(0).detach().cpu() - test_model.layer_out.bias.data - torch.relu(test_model.layer_out.weight.data).sum().detach().cpu()

tensor([-0.3933])

In [92]:
torch.relu(test_model.layer_out.weight.data).sum().detach().cpu()

tensor(0.3711)

In [96]:
test_model.layer_out.weight.data

tensor([[-0.3023,  0.3711]], device='cuda:0')

In [75]:
torch.relu(test_model.layer_out.weight.data).sum()

tensor(0.3711, device='cuda:0')

In [4]:
model = train(train_loader=train_loader, val_loader=val_loader, model=test_model, epochs=100, optimizer=optimizer, criterion=criterion, device=device)

[ 1/100 ] | train_loss = 0.76017, train_acc = 0.55777, val_loss = 0.75389, val_acc = 0.51852
[ 2/100 ] | train_loss = 0.75716, train_acc = 0.55386, val_loss = 0.75074, val_acc = 0.51852
[ 3/100 ] | train_loss = 0.75419, train_acc = 0.55974, val_loss = 0.74764, val_acc = 0.51852
[ 4/100 ] | train_loss = 0.75122, train_acc = 0.56756, val_loss = 0.74459, val_acc = 0.51852
[ 5/100 ] | train_loss = 0.74820, train_acc = 0.57932, val_loss = 0.74151, val_acc = 0.51852
[ 6/100 ] | train_loss = 0.74517, train_acc = 0.57344, val_loss = 0.73846, val_acc = 0.53704
[ 7/100 ] | train_loss = 0.74221, train_acc = 0.58911, val_loss = 0.73554, val_acc = 0.53704
[ 8/100 ] | train_loss = 0.73941, train_acc = 0.59499, val_loss = 0.73270, val_acc = 0.53704
[ 9/100 ] | train_loss = 0.73666, train_acc = 0.59890, val_loss = 0.72997, val_acc = 0.53704
[ 10/100 ] | train_loss = 0.73398, train_acc = 0.60280, val_loss = 0.72725, val_acc = 0.53704
[ 11/100 ] | train_loss = 0.73121, train_acc = 0.61062, val_loss = 0.